
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022
    
Adapted by J. Orjuela (DIADE-IRD) - mai 2023


# <span style="color:#006E7F">__TP3 - CONTIGS QUALITY__ <a class="anchor" id="data"></span>  
    
# <span style="color: #4CACBC;"> 1. Perform a comparison of assemblies using QUAST</span>  


The metrics of assemblies can be evaluated using a quality assessment tool such as [QUAST](http://quast.bioinf.spbau.ru/manual.html).

In part 2, we have assembled raw data using various assemblers and also polished and corrected it. 

Aggregate all assemblies you have produced for your sample in a new folder named "AGGREGATED".

We're going to compare it with QUAST.

In [1]:
mkdir -p ~/work/RESULTS/AGGREGATED
cd ~/work/RESULTS/AGGREGATED

### Order assemblies obtained in the samples 

#### sample 4222

In [2]:
ln -s ~/work/RESULTS/4222_FLYE/assembly.fasta 4222_FLYE.fasta
ln -s ~/work/RESULTS/4222_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/4222_RAVEN/assembly.fasta 4222_RAVEN.fasta
ln -s ~/work/RESULTS/4222_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_RAVEN_MEDAKA.fasta

#### sample B8

In [3]:
ln -s ~/work/RESULTS/B8_FLYE/assembly.fasta B8_FLYE.fasta
ln -s ~/work/RESULTS/B8_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta B8_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/B8_RAVEN/assembly.fasta B8_RAVEN.fasta
ln -s ~/work/RESULTS/B8_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta B8_RAVEN_MEDAKA.fasta

#### sample G11

In [4]:
ln -s ~/work/RESULTS/G11_FLYE/assembly.fasta G11_FLYE.fasta
ln -s ~/work/RESULTS/G11_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta G11_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/G11_RAVEN/assembly.fasta G11_RAVEN.fasta
ln -s ~/work/RESULTS/G11_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta G11_RAVEN_MEDAKA.fasta

#### check symbolic links

In [5]:
ls -l ~/work/RESULTS/AGGREGATED

total 24
lrwxrwxrwx 1 jovyan users 50 Oct 29 12:40 4222_FLYE.fasta -> /home/jovyan/work/RESULTS/4222_FLYE/assembly.fasta
lrwxrwxrwx 1 jovyan users 75 Oct 29 12:40 4222_FLYE_MEDAKA.fasta -> /home/jovyan/work/RESULTS/4222_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta
lrwxrwxrwx 1 jovyan users 51 Oct 29 12:40 4222_RAVEN.fasta -> /home/jovyan/work/RESULTS/4222_RAVEN/assembly.fasta
lrwxrwxrwx 1 jovyan users 76 Oct 29 12:40 4222_RAVEN_MEDAKA.fasta -> /home/jovyan/work/RESULTS/4222_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta
lrwxrwxrwx 1 jovyan users 48 Oct 29 12:40 B8_FLYE.fasta -> /home/jovyan/work/RESULTS/B8_FLYE/assembly.fasta
lrwxrwxrwx 1 jovyan users 73 Oct 29 12:40 B8_FLYE_MEDAKA.fasta -> /home/jovyan/work/RESULTS/B8_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta
lrwxrwxrwx 1 jovyan users 49 Oct 29 12:40 B8_RAVEN.fasta -> /home/jovyan/work/RESULTS/B8_RAVEN/assembly.fasta
lrwxrwxrwx 1 jovyan users 74 Oct 29 12:40 B8_RAVEN_MEDAKA.fasta -> /home/jovyan/work/RESULTS/B8_RAVEN_MEDAKA/MEDAKA_C

### <span style="color: #4CACBD;"> 1.1  Run QUAST on assemblies and compare them </span>

In [7]:
cd ~/work/RESULTS/AGGREGATED/
time quast.py *.fasta -o QUAST

/opt/conda/envs/quast/bin/quast.py 4222_FLYE.fasta 4222_FLYE_MEDAKA.fasta 4222_RAVEN.fasta 4222_RAVEN_MEDAKA.fasta B8_FLYE.fasta B8_FLYE_MEDAKA.fasta B8_RAVEN.fasta B8_RAVEN_MEDAKA.fasta G11_FLYE.fasta G11_FLYE_MEDAKA.fasta G11_RAVEN.fasta G11_RAVEN_MEDAKA.fasta -o QUAST

Version: 5.2.0

System information:
  OS: Linux-4.19.0-25-cloud-amd64-x86_64-with-debian-bookworm-sid (linux_64)
  Python version: 3.7.12
  CPUs number: 8

Started: 2023-10-29 19:22:48

Logging to /home/jovyan/work/RESULTS/AGGREGATED/QUAST/quast.log
NOTICE: Maximum number of threads is set to 2 (use --threads option to set it manually)

CWD: /home/jovyan/work/RESULTS/AGGREGATED
Main parameters: 
  MODE: default, threads: 2, min contig length: 500, min alignment length: 65, min alignment IDY: 95.0, \
  ambiguity: one, min local misassembly length: 200, min extensive misassembly length: 1000

Contigs:
  Pre-processing...
  1  4222_FLYE.fasta ==> 4222_FLYE
  2  4222_FLYE_MEDAKA.fasta ==> 4222_FLYE_MEDAKA
  3  4222_RAVEN.

#### Go to the QUAST directory and check the file content.

#### Looking at the output statistics, what are the main differences between the assemblies?

#### Look total sizes, number of contigs, N50 statistics.

#### For each sample what is the best one ?

### <span style="color: #4CACBD;"> 1.2  Comparison of the assemblies against the reference genome of closely organism </span>

It's possible to compare assemblies against the reference genome of closely related organism. 

Use corrected assemblies and compare it with the reference genome

### Compare best algae assemblies against the reference sequence using QUAST

create for that a QUAST_REF directory into the AGGREGATED one

#### Which of the assemblies are closest in size relative to the reference ?

#### Does this have the largest N50 and fewest number of contigs?

#### What assembly does obtain the best contiguity metrics?


## <span style="color: #4CACBD;"> 2. Assessing gene space using BUSCO </span>


Benchmarking Universal Single-Copy Orthologs ([BUSCO](https://busco.ezlab.org/busco_userguide.html)) helps to check if you have a good assembly, by searching the expected single-copy lineage-conserved orthologs in any newly-sequenced genome from an appropriate phylogenetic clade.

Calculate the gene space for each assembly

In [8]:
mkdir -p ~/work/RESULTS/AGGREGATED/BUSCO
cd ~/work/RESULTS/AGGREGATED/BUSCO

Lineage can be choose in busco database using --list-datasets parameter

In [9]:
# busco env
conda activate busco

In [10]:
busco --list-datasets

2023-10-29 19:24:37 INFO:	Downloading information on latest versions of BUSCO data...
2023-10-29 19:24:40 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/information/lineages_list.2021-12-14.txt.tar.gz'
2023-10-29 19:24:41 INFO:	Decompressing file '/home/jovyan/work/RESULTS/AGGREGATED/BUSCO/busco_downloads/information/lineages_list.2021-12-14.txt.tar.gz'

################################################

Datasets available to be used with BUSCO v4 and v5:

 bacteria_odb10
     - acidobacteria_odb10
     - actinobacteria_phylum_odb10
         - actinobacteria_class_odb10
             - corynebacteriales_odb10
             - micrococcales_odb10
             - propionibacteriales_odb10
             - streptomycetales_odb10
             - streptosporangiales_odb10
         - coriobacteriia_odb10
             - coriobacteriales_odb10
     - aquificae_odb10
     - bacteroidetes-chlorobi_group_odb10
         - bacteroidetes_odb10
             - bacteroidia_odb10
                 

In [11]:
ASSEMBLY="/home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta"
LINEAGE=chlorophyta_odb10
busco -i "$ASSEMBLY" -l "$LINEAGE" -c "${CPUS:-8}" -m genome -o 4222_FLYE_MEDAKA_BUSCO 
# deactivate busco env
conda deactivate

2023-10-29 19:24:55 INFO:	***** Start a BUSCO v5.5.0 analysis, current time: 10/29/2023 19:24:55 *****
2023-10-29 19:24:55 INFO:	Configuring BUSCO with local environment
2023-10-29 19:24:55 INFO:	Mode is genome
2023-10-29 19:24:55 INFO:	Downloading information on latest versions of BUSCO data...
2023-10-29 19:25:00 INFO:	Input file is /home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta
2023-10-29 19:25:00 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/chlorophyta_odb10.2020-08-05.tar.gz'
2023-10-29 19:25:03 INFO:	Decompressing file '/home/jovyan/work/RESULTS/AGGREGATED/BUSCO/busco_downloads/lineages/chlorophyta_odb10.tar.gz'
2023-10-29 19:25:08 INFO:	Running BUSCO using lineage dataset chlorophyta_odb10 (eukaryota, 2020-08-05)
2023-10-29 19:25:08 INFO:	Running 1 job(s) on bbtools, starting at 10/29/2023 19:25:08
2023-10-29 19:25:10 INFO:	[bbtools]	1 of 1 task(s) completed
2023-10-29 19:25:11 INFO:	Running 1 job(s) on metaeuk, starting at 10/29/2023 19:25:

### What does the gene space look like for this assembly?

### Run BUSCO for the best assemblies ...

## <span style="color: #4CACBD;"> 3. Read alignment statistics - remapping</span>

Read congruency is an important measure in determining assembly accuracy. Clusters of read pairs that align incorrectly are strong indicators of mis-assembly.

How well do the reads align back to the draft assemblies? Use minimap2 and samtools to assess the basic alignment statistics.

Make a folder for your results.

We will use 4222_FLYE_MEDAKA.fasta assembly and ONT READS as well ILLUMINA READS.

In [12]:
mkdir -p ~/work/RESULTS/REMAPPING
cd ~/work/RESULTS/REMAPPING/

In [13]:
ASSEMBLY="/home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta"
ONT="/home/jovyan/work/DATA/4222_RB2.fastq.gz"

In [14]:
#symbolic link from last assembly in the current repertory REMAPPING
ln -s ${ASSEMBLY} 4222_FLYE_MEDAKA.fasta
# overwrite ASSEMBLY variable
ASSEMBLY=4222_FLYE_MEDAKA.fasta

## Mapping on assemblies with ONT reads

In [15]:
minimap2 -ax map-ont -t 4 ${ASSEMBLY} ${ONT} | samtools sort -@ 1 -T "${ASSEMBLY/.fasta/}" -O BAM -o "${ASSEMBLY/.fasta/_ONT_minimap2.bam}" -
samtools index "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"
samtools view -F 0x904 -c "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"

[M::mm_idx_gen::1.618*1.00] collected minimizers
[M::mm_idx_gen::1.883*1.38] sorted minimizers
[M::main::1.883*1.38] loaded/built the index for 31 target sequence(s)
[M::mm_mapopt_update::2.030*1.35] mid_occ = 21
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 31
[M::mm_idx_stat::2.099*1.34] distinct minimizers: 3864622 (94.31% are singletons); average occurrences: 1.099; average spacing: 5.330; total length: 22632780
[M::worker_pipeline::141.084*3.82] mapped 42406 sequences
[M::worker_pipeline::149.335*3.79] mapped 20357 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax map-ont -t 4 4222_FLYE_MEDAKA.fasta /home/jovyan/work/DATA/4222_RB2.fastq.gz
[M::main] Real time: 149.390 sec; CPU: 566.427 sec; Peak RSS: 2.710 GB
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
62394


### What is the percentage of aligned ONT reads in your clone assembly?

## <span style="color: #4CACBD;"> [OPTIONAL] Blobtools </span>

During the sequence quality assessment stage we tried to discern whether contamination was present. Sometimes this is not feasible at the read level. By plotting Contig GC content vs Contig Read Coverage we can look for clusters of contigs that share similar coverage. The appearance of multiple clusters can indicate multiple organisms. Occasionally, contigs can also be taxonomically classified, providing further evidence for contaminants.

First we need to download some files

In [44]:
cd ~/work/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/blobtools.tar.gz
tar zxvf blobtools.tar.gz
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/testBacteria.dmnd

--2023-10-30 16:06:28--  https://itrop.ird.fr/ont-training/blobtools.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100114880 (95M) [application/x-gzip]
Saving to: ‘blobtools.tar.gz’

blobtools.tar.gz    100%[===================>]  95.48M  51.4MB/s    in 1.9s    

2023-10-30 16:06:30 (51.4 MB/s) - ‘blobtools.tar.gz’ saved [100114880/100114880]

FINISHED --2023-10-30 16:06:30--
Total wall clock time: 2.3s
Downloaded: 1 files, 95M in 1.9s (51.4 MB/s)
blobtools/
blobtools/taxdump.tar.gz
blobtools/names.dmp
blobtools/nodes.dmp
--2023-10-30 16:06:34--  https://itrop.ird.fr/ont-training/testBacteria.dmnd
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214010272 (204M)
Saving to: ‘testBacteria.dmnd’

testBacteria.dm

Run Blobtools on each assembly. Blobtools requires both a BAM file as input and blast output for the classification step.2.

In [51]:
mkdir -p ~/work/RESULTS/BLOBTOOLS
cd ~/work/RESULTS/BLOBTOOLS

### Blastx using diamond 

Run diamond in blastx mode using assembled contigs vs a pre-formatted diamond bacteria database (protein)

In [18]:
#symbolic link from last assembly in the current repertory BLOBTOOLs
ln -s ~/work/RESULTS/4222_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_FLYE_MEDAKA.fasta

In [19]:
time diamond blastx --query 4222_FLYE_MEDAKA.fasta --db ~/work/DATA/testBacteria.dmnd --outfmt 6 qseqid staxids bitscore qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore --sensitive  --max-target-seqs 1  --evalue 1e-25  --threads 4  --out diamond.csv

diamond v2.1.8.162 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
#Target sequences to report alignments for: 1
Opening the database...  [0.095s]
Database: /home/jovyan/work/DATA/testBacteria.dmnd (type: Diamond database, sequences: 271272, letters: 174523808)
Block size = 2000000000
Opening the input file...  [0.035s]
Opening the output file...  [0s]
Loading query sequences...  [0.508s]
Masking queries...  [1.611s]
Algorithm: Double-indexed
Building query histograms...  [2.745s]
Seeking in database...  [0s]
Loading reference sequences...  [0.757s]
Masking reference...  [3.797s]
Initializing temporary storage...  [0s]
Building reference histograms...  [7.731s]
Allocating

In [20]:
head diamond.csv

contig_16	217031	369	contig_16	tr|A0A177ZJ03|A0A177ZJ03_9BACI	58.6	333	132	4	25349	24354	4	331	5.09e-113	369
contig_19	1176587	947	contig_19	tr|A0A1A9I5F8|A0A1A9I5F8_9BACT	45.5	1185	584	15	515616	519113	13	1154	9.80e-296	947
contig_14	1822215	530	contig_14	tr|A0A165Q899|A0A165Q899_9SPHN	48.1	642	218	4	40879	38954	2	528	5.94e-166	530
contig_4	1727196	983	contig_4	tr|A0A172U9I0|A0A172U9I0_9GAMM	74.1	652	168	1	520369	518414	2	652	1.01e-317	983
contig_23	879274	526	contig_23	tr|A0A1A9FY51|A0A1A9FY51_9RHIZ	49.4	597	280	7	10953	9223	7	601	1.13e-163	526
contig_12	445710	315	contig_12	tr|A0A160MXZ8|A0A160MXZ8_9GAMM	44.3	345	183	5	29798	28767	23	359	3.37e-94	315
contig_33	1848039	712	contig_33	tr|A0A1A9NP23|A0A1A9NP23_9PROT	73.4	463	123	0	60166	58778	6	468	1.86e-230	712
contig_8	161355	302	contig_8	tr|A0A171DEZ1|A0A171DEZ1_9ACTN	34.6	540	312	8	5843	4275	161	676	6.40e-82	302
contig_27	1727196	658	contig_27	tr|A0A172UCM3|A0A172UCM3_9GAMM	40.8	926	476	18	542190	544913	1	872	1.07e-203	658
contig_5	

In [52]:
pwd

/home/jovyan/work/RESULTS/BLOBTOOLS


### Run blobtools 

Specifically to Blobtools: * nodes and names from ncbi taxdump database can be download from [here](https://github.com/DRL/blobtools#download-ncbi-taxdump-and-create-nodesdb).

In this training nodes and names are available on the DATA directory

In [53]:
ASSEMBLY="4222_FLYE_MEDAKA.fasta"
BAM=~/work/RESULTS/REMAPPING/${ASSEMBLY/.fasta/_ONT_minimap2.bam}
DIAMONDX=~/work/RESULTS/BLOBTOOLS/diamond.csv
BLOB_NODES=~/work/DATA/blobtools/nodes.dmp
BLOB_NAMES=~/work/DATA/blobtools/names.dmp

In [55]:
blobtools create -i ${ASSEMBLY} -b ${BAM} -t ${DIAMONDX} -o quality --names ${BLOB_NAMES} --nodes ${BLOB_NODES} --db nodesDB.txt

[+] Parsing FASTA - 4222_FLYE_MEDAKA.fasta
[+] Store nodesDB in nodesDB.txt
[%] : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.33M/2.33M [00:04<00:00, 517kit/s]
[+] Parsing tax0 - /home/jovyan/work/RESULTS/BLOBTOOLS/diamond.csv
[+] Computing taxonomy using taxrule(s) bestsum
[%] : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31.0/31.0 [00:00<00:00, 27.2kit/s]
[+] Parsing bam0 - /home/jovyan/work/RESULTS/REMAPPING/4222_FLYE_MEDAKA_ONT_minimap2.bam
[+] -> 93.55 (29/31) of sequences have reads aligned to them.
[+] -> 99.77 (163193/163562) of reads are mapped.
Traceback (most recent call last):
  File "/opt/conda/envs/blobtools/bin/blobtools", line 10, in <module>
    sys.exit(main())
  File "/opt/conda/envs/blobtools/lib/python3.8/site-packages/blobtools/interfa

: 1

In [47]:
blobtools view -i quality.blobDB.json --cov -o output;
blobtools plot -i quality.blobDB.json;

[ERROR:0]	: File quality.blobDB.json does not exist.
[+] Reading BlobDB quality.blobDB.json
[ERROR:0]	: File quality.blobDB.json does not exist.


: 1

#### Is there contamination in the assembly?

#### Do any assemblies show strange clustering?

#### Why might coverage vary across contigs within an assembly?

The Blobplots all indicate a single cluster. Some contigs show fairly high coverage in comparison to the rest of the genome which could be repetitive elements in the genome.